In [123]:
from butterfly.blockMeshDict import BlockMeshDict
from butterfly.snappyHexMeshDict import SnappyHexMeshDict


In [124]:
# [ ] Create snappyHexMeshDict file from annotation
# [ ] Convert result from OpenFOAM to SU2 format
# [ ] Run SU2 simulation
# [ ] Run OpenFoam simulation later as well

out_dir = "/workspaces/butterfly/examples/out"

In [152]:
import cadquery as cq

subject_profile = cq.importers.importStep(f"{out_dir}/CNCRD.STEP").rotate((0, 0, 0), (0, 0, 1), 180).rotate((0, 0, 0), (1, 0, 0), -90)

subject_solid = subject_profile.val()
subject_bbox = subject_solid.BoundingBox()

# cq.exporters.export(subject_profile, f"{out_dir}/aircraft.stl")
cq.Shape.exportStl(subject_profile.val(), f"{out_dir}/constant/geometry/aircraft.stl", ascii=True)
# cq.exporters.export(wind_tunnel_profile, f"{out_dir}/wind_tunnel.stl")


True

In [153]:
subject_center = subject_solid.Center()

wind_tunnel_size = (subject_bbox.xlen*8, subject_bbox.ylen*15, subject_bbox.zlen*30)


refinement_size = (subject_bbox.xlen*2, subject_bbox.ylen*3, subject_bbox.zlen*5)


# wind_tunnel_profile = (
#     cq.Workplane("XY", origin=[subject_center.x-wind_tunnel_size[0]*0.2, subject_center.y, subject_center.z])
#     .box(*wind_tunnel_size)    
# )


refinement_box_profile = (
    cq.Workplane("XY", origin=[subject_center.x-refinement_size[0]*0.15, subject_center.y, subject_center.z])
    .box(*refinement_size)
)

refinement_bbox = refinement_box_profile.val().BoundingBox()
refinement_min = (refinement_bbox.xmin, refinement_bbox.ymin, refinement_bbox.zmin)
refinement_max = (refinement_bbox.xmax, refinement_bbox.ymax, refinement_bbox.zmax)

print(refinement_min)
print(refinement_max)
print(subject_center + cq.Vector(0, 0, subject_bbox.zlen*2))

(-118.1544649081838, -38.101175904444545, -12.41767296521679)
(4.948264830918582, 38.098824695555294, 34.86939031518263)
Vector: (-38.13769057776725, -0.0011756044446253617, 30.140683987142687)


In [149]:
from jupyter_cadquery import show

show(subject_profile, refinement_box_profile)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.01s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [110]:
# from jupyter_cadquery import show

inlet_faces = wind_tunnel_profile.faces(">X").vals()
outlet_faces = wind_tunnel_profile.faces("<X").vals()
side_faces = wind_tunnel_profile.faces("<Y or >Y").vals()
top_faces = wind_tunnel_profile.faces(">Z").vals()
bottom_faces = wind_tunnel_profile.faces("<Z").vals()

# show(wind_tunnel_profile.faces(">Z"))

In [120]:
import numpy as np
from butterfly.boundarycondition import BoundaryCondition, FixedInletBoundaryCondition, FixedOutletBoundaryCondition, WindTunnelGroundBoundaryCondition, WindTunnelInletBoundaryCondition, WindTunnelOutletBoundaryCondition, WindTunnelTopAndSidesBoundaryCondition, WindTunnelWallBoundaryCondition
from butterfly.conditions import ABLConditions
from butterfly.geometry import BFBlockGeometry, BFGeometry, bf_geometry_from_stl_file
from butterfly.windtunnel import WindTunnel


abc = ABLConditions()

def get_geo(label: str, face: cq.Face, bc: BoundaryCondition):
    cq_vertices, cq_indices = face.tessellate(0.01)
    vertices = np.array([v.toTuple() for v in cq_vertices])
    indices = np.array(cq_indices)
    border_vertices = [[v.toTuple() for v in face.Vertices()]]
    return BFBlockGeometry(label, vertices.tolist(), indices.tolist(), border_vertices, bc)



def get_tri_normal(tri_vertices: np.ndarray):
    # Calculate two vectors along the edges of the triangle
    normal = np.cross(
        tri_vertices[1] - tri_vertices[0], tri_vertices[2] - tri_vertices[0]
    )
    unit_normal = normal / np.linalg.norm(normal, axis=0, keepdims=True)
    return unit_normal


def get_test_geo(name: str, solid: cq.Solid):
    cq_vertices, cq_indices = solid.tessellate(0.01)

    tri_vertices = np.array([v.toTuple() for v in cq_vertices])
    tri_indices = np.array(cq_indices)
    tri_normals = np.array([get_tri_normal(tri_vertices[tri]) for tri in tri_indices])
    return BFGeometry(name, tri_vertices.tolist(), tri_indices.tolist(), tri_normals.tolist())

inlet_bc = WindTunnelInletBoundaryCondition(abc)
inlet_geo = get_geo("inlet", inlet_faces[0], inlet_bc)


outlet_bc = WindTunnelOutletBoundaryCondition()
outlet_geo = get_geo("outlet", outlet_faces[0], outlet_bc)

sides_bc = WindTunnelTopAndSidesBoundaryCondition()
side_geos = [
    get_geo("left", side_faces[0], sides_bc),
    get_geo("right", side_faces[1], sides_bc),
]

top_bc = WindTunnelTopAndSidesBoundaryCondition()
top_geo = get_geo("top", top_faces[0], top_bc)


bottom_bc = WindTunnelGroundBoundaryCondition(abc)
bottom_geo = get_geo("bottom", bottom_faces[0], bottom_bc)


test_geos = [
    # get_test_geo("plane", subject_solid)
]
bounding_geometries = [inlet_geo, outlet_geo, *side_geos, top_geo, bottom_geo]

wind_tunnel = WindTunnel("wind_tunnel", inlet_geo, outlet_geo, side_geos, top_geo, bottom_geo, test_geos, roughness=0.005)
wind_tunnel.blockMeshDict.n_div_xyz = (int(0.06*wind_tunnel_size[0]), int(0.15*wind_tunnel_size[1]), int(0.15*wind_tunnel_size[2]))


# (50, 30, 15)




In [72]:
blockMeshDict = BlockMeshDict.from_bf_block_geometries(bounding_geometries)

In [121]:
# with open(f"{out_dir}/blockMeshDict", "w") as f:
#     f.write(wind_tunnel.blockMeshDict.to_openfoam())
case = wind_tunnel.to_openfoam_case()

In [122]:
from pathlib import Path
for file in case.foam_files:
    case_dir = Path(f"{out_dir}/{file.location.replace('"', "")}")
    case_dir.mkdir(parents=True, exist_ok=True)
    with open(f"{case_dir}/{file.name}", "w") as f:
        f.write(str(file))

In [2]:
snappyHexMesh = SnappyHexMeshDict(
    {
        "castellatedMeshControls": {
            "nCellsBetweenLevels": 3,  # "5-10 at least", 3 is at the lower end
            # "refinementSurfaces": {
            #     "motorBike": {
            #         "level": (6, 8),  # "refined 6-8", matches the comment, assumed non-default
            #     }
            # },
            # amount of refinement based on cell angles (30, 45)
            "resolveFeatureAngle": 180,  # Default not specified, likely non-default
            # "refinementRegions": {
            #     "refinementBox": {
            #         "mode": "inside",  # Mode specified, assumed non-default
            #         "level": 5,  # Level specified, assumed non-default
            #     }
            # },
            # external then use surface projection, if interanl (pipe) use interior point (com)

            "locationInMesh": (0, 0, 0),  # No default mentioned, assumed non-default
            "allowFreeStandingZoneFaces": True,  # Default not specified, likely non-default
        },
        "snapControls": {
            "tolerance": 5,  # "1~4 is typical", 5 is above typical
            "nSolveIter": 100,  # "100 or 150 is sweet spot", matches the comment
            "implicitFeatureSnap": False,  # Default is false, matches the comment
            "explicitFeatureSnap": True,  # Default is true, matches the comment
        },
        "addLayersControls": {
            # "relativeSizes": True,  # Default not specified, likely non-default
            # "layers": {
            #     "motorBike": {
            #         "nSurfaceLayers": 3,  # "minimum 1 is expected", assumed non-default
            #     }
            # },
            "expansionRatio": 1.0,  # "1.2~1.5", 1.0 is below the typical range
            "minThickness": 0.2,  # "0.01~0.1", 0.2 is above the range
            "featureAngle": 110,  # "130~170", 110 is below the range
        },
        "meshQualityControls": {
            "maxNonOrtho": 65,  # Default is 65, 60 is lower (better)
            "minTwist": 0.05,  # Default is 0.05, 0.02 is stricter
            "minFaceWeight": 0.05,  # Default is 0.05, 0.02 is stricter
        },

    }
)



In [3]:
out_dir = "/workspaces/butterfly/examples/out"
with open(f"{out_dir}/snappyHexMeshDict", "w") as f:
    f.write(snappyHexMesh.to_openfoam())